In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Import the dataset
Dataset download link: https://www.kaggle.com/datasets/kartik2112/fraud-detection/data

The dataset is saved in two files. This dataset has no missing values.

In [2]:
dataset_train = pd.read_csv('fraudTrain.csv')
dataset_test = pd.read_csv('fraudTest.csv')
dataset = pd.concat([dataset_train, dataset_test])
dataset.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1852394 entries, 0 to 555718
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   trans_date_trans_time  object 
 2   cc_num                 int64  
 3   merchant               object 
 4   category               object 
 5   amt                    float64
 6   first                  object 
 7   last                   object 
 8   gender                 object 
 9   street                 object 
 10  city                   object 
 11  state                  object 
 12  zip                    int64  
 13  lat                    float64
 14  long                   float64
 15  city_pop               int64  
 16  job                    object 
 17  dob                    object 
 18  trans_num              object 
 19  unix_time              int64  
 20  merch_lat              float64
 21  merch_long             float64
 22  is_fraud           

# Drop redundant data from the dataset:
- The first unnamed column and 'trans_num' are arbitrarily represetations of each transaction.
- Data in 'unix_time' (The time in which a transaction occurred in unix time) is already provided by 'trans_date_trans_time'.
- Each unique 'cc_num' (credit card number) represents unique credit card whose owner is a person. Said person can be identified by a combination of their data in 'lat' and 'long' (coordinates), 'dob' (date of birth) and 'job'. Other personal data that does not contain any other meaning apart from that of identification therefore can be dropped, namely 'first', 'last' (names), 'street', 'state', 'zip' ,'city' (address).

In [4]:
dataset['cc_num'].nunique()

999

In [5]:
dataset.groupby(['long', 'lat', 'job', 'dob']).nunique()

,,,,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,city_pop,trans_num,unix_time,merch_lat,merch_long,is_fraud
long,lat,job,dob,,,,,,,,,,,,,,,,,,,
-165.6723,64.7556,"Administrator, education",1939-11-09,2203,2203,2203,2203,2203,2203,2203,2203,2203,2203,2203,2203,2203,2203,2203,2203,2203,2203,2203
-156.2920,65.6899,"Engineer, civil (consulting)",1965-11-06,734,734,734,734,734,734,734,734,734,734,734,734,734,734,734,734,734,734,734
-155.4880,20.0827,Physiotherapist,1966-12-03,1463,1463,1463,1463,1463,1463,1463,1463,1463,1463,1463,1463,1463,1463,1463,1463,1463,1463,1463
-155.3697,20.0271,"Scientist, research (maths)",1991-04-13,2186,2186,2186,2186,2186,2186,2186,2186,2186,2186,2186,2186,2186,2186,2186,2186,2186,2186,2186
-153.9940,66.6933,Careers information officer,1996-04-01,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-69.9576,44.3229,Advertising account planner,1995-04-22,737,737,737,737,737,737,737,737,737,737,737,737,737,737,737,737,737,737,737
-69.4828,44.0943,Financial trader,1976-04-12,2931,2931,2931,2931,2931,2931,2931,2931,2931,2931,2931,2931,2931,2931,2931,2931,2931,2931,2931
-69.2675,44.8393,Lexicographer,1995-12-28,743,743,743,743,743,743,743,743,743,743,743,743,743,743,743,743,743,743,743


In [6]:
cols_drop = ['unix_time', 'first', 'last', 'street', 'city', 'state', 'zip', 'trans_num', 'cc_num']
dataset_dropped = dataset

dataset_dropped = dataset_dropped.drop(dataset_test.columns[0], axis = 1)
dataset_dropped = dataset_dropped.drop(columns = cols_drop)

dataset_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1852394 entries, 0 to 555718
Data columns (total 13 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trans_date_trans_time  object 
 1   merchant               object 
 2   category               object 
 3   amt                    float64
 4   gender                 object 
 5   lat                    float64
 6   long                   float64
 7   city_pop               int64  
 8   job                    object 
 9   dob                    object 
 10  merch_lat              float64
 11  merch_long             float64
 12  is_fraud               int64  
dtypes: float64(5), int64(2), object(6)
memory usage: 197.9+ MB


In [7]:
X = dataset_dropped.iloc[:, :-1]
X

,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,dob,merch_lat,merch_long
0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,36.011293,-82.048315
1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,49.159047,-118.186462
2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,43.150704,-112.154481
3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,Patent attorney,1967-01-12,47.034331,-112.561071
4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,38.674999,-78.632459
...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,fraud_Reilly and Sons,health_fitness,43.77,M,40.4931,-91.8912,519,Town planner,1966-02-13,39.946837,-91.333331
555715,2020-12-31 23:59:09,fraud_Hoppe-Parisian,kids_pets,111.84,M,29.0393,-95.4401,28739,Futures trader,1999-12-27,29.661049,-96.186633
555716,2020-12-31 23:59:15,fraud_Rau-Robel,kids_pets,86.88,F,46.1966,-118.9017,3684,Musician,1981-11-29,46.658340,-119.715054
555717,2020-12-31 23:59:24,fraud_Breitenberg LLC,travel,7.99,M,44.6255,-116.4493,129,Cartographer,1965-12-15,44.470525,-117.080888


In [8]:
y = dataset_dropped.iloc[:, -1]
y

0         0
1         0
2         0
3         0
4         0
         ..
555714    0
555715    0
555716    0
555717    0
555718    0
Name: is_fraud, Length: 1852394, dtype: int64

# Transform data
We want to transform all data to numerical values so it can fit in with any model, and the range is between 0 and 1 to avoid bias.

## Numerical data

### Date of birth
Convert date of birth to age (relative to the time of transaction).

In [9]:
from datetime import datetime

def calculate_age(row):
    trans_date = datetime.strptime(row['trans_date_trans_time'], '%Y-%m-%d %H:%M:%S')
    date_of_birth = datetime.strptime(row['dob'], '%Y-%m-%d')
    return trans_date.year - date_of_birth.year

X['age'] = X.apply(calculate_age, axis = 1)

In [10]:
X = X.drop('dob', axis = 1)
X.head()

,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age
0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,"Psychologist, counselling",36.011293,-82.048315,31
1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,Special educational needs teacher,49.159047,-118.186462,41
2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,Nature conservation officer,43.150704,-112.154481,57
3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,Patent attorney,47.034331,-112.561071,52
4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,Dance movement psychotherapist,38.674999,-78.632459,33


In the United States, generally you have to be 18 or older to be authorized a credit card. On the other end of the spectrum, while it's hard to know who is the oldest person in the country to own a credit card, it's known that the oldest living person is 114 years old. We will use these two numbers to scale the age column of the data frame with a custom version of MaxMinScaler.

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class AgeScaler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        min_age = 18
        max_age = 114
        return (X - min_age) / (max_age - min_age)

X['age'] = AgeScaler().fit_transform(X['age'])

In [12]:
X.head()

,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age
0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,"Psychologist, counselling",36.011293,-82.048315,0.135417
1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,Special educational needs teacher,49.159047,-118.186462,0.239583
2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,Nature conservation officer,43.150704,-112.154481,0.406250
3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,Patent attorney,47.034331,-112.561071,0.354167
4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,Dance movement psychotherapist,38.674999,-78.632459,0.156250


### Coordinates
Similar to customers' age data, we use the coordinate boundaries of the US to scale the our coordinates as following:
- Northernmost (lat_max): 49.382808
- Southernmost (lat_min): 24.521208
- Easternmost (long_max): -66.945392
- Westernmost (long_min): -124.736342

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin

class LatitudeScaler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        lat_max = 49.382808
        lat_min = 24.521208
        return (X - lat_min) / (lat_max - lat_min)
class LongitudeScaler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        long_max = -66.945392
        long_min = -124.736342
        return (X - long_min) / (long_max - long_min)

X['lat'] = LatitudeScaler().fit_transform(X['lat'])
X['long'] = LongitudeScaler().fit_transform(X['long'])
X['merch_lat'] = LatitudeScaler().fit_transform(X['merch_lat'])
X['merch_long'] = LongitudeScaler().fit_transform(X['merch_long'])
X.head()

,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age
0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,0.464877,0.753721,3495,"Psychologist, counselling",0.462162,0.738663,0.135417
1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,0.980089,0.112922,149,Special educational needs teacher,0.991000,0.113337,0.239583
2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,M,0.710316,0.215853,4154,Nature conservation officer,0.749328,0.217713,0.406250
3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,0.873210,0.218417,1939,Patent attorney,0.905538,0.210678,0.354167
4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,M,0.559075,0.783400,99,Dance movement psychotherapist,0.569303,0.797770,0.156250


### City population
The most popular city in the US in 2020 is New York City at 8,804,190 people which can be used similarly to the above. However, this dataset was generated in a simulation of which the city population does not match with real life. Therefore we will just use the standard MinMaxScaler for this one.

In [14]:
dataset.groupby(['city_pop', 'city']).nunique()

,,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,zip,lat,long,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
city_pop,city,,,,,,,,,,,,,,,,,,,,,
23,Notrees,2912,2915,1,643,14,2510,1,1,1,1,...,1,1,1,1,1,2915,2915,2913,2909,2
37,Gregory,1469,1469,1,572,14,1357,1,1,1,1,...,1,1,1,1,1,1469,1469,1469,1469,2
43,Ratcliff,2918,2920,1,642,14,2440,1,1,1,1,...,1,1,1,1,1,2920,2920,2919,2920,1
46,Thompson,4381,4386,1,684,14,3326,1,1,1,1,...,1,1,1,1,1,4386,4386,4381,4381,2
47,West Bethel,734,734,1,427,14,703,1,1,1,1,...,1,1,1,1,1,734,734,734,734,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577385,New York City,3679,3680,4,682,14,3150,4,4,1,4,...,4,4,4,4,4,3680,3680,3678,3678,2
1595797,San Antonio,7302,7312,3,693,14,4726,3,3,2,3,...,3,3,3,3,3,7312,7312,7301,7297,2
2383912,Los Angeles,737,737,1,448,14,709,1,1,1,1,...,1,1,1,1,1,737,737,737,737,2


In [15]:
from sklearn.preprocessing import MinMaxScaler
X['city_pop'] = MinMaxScaler().fit_transform(X[['city_pop']])

X.head()

,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age
0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,0.464877,0.753721,0.001194,"Psychologist, counselling",0.462162,0.738663,0.135417
1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,0.980089,0.112922,0.000043,Special educational needs teacher,0.991000,0.113337,0.239583
2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,M,0.710316,0.215853,0.001421,Nature conservation officer,0.749328,0.217713,0.406250
3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,0.873210,0.218417,0.000659,Patent attorney,0.905538,0.210678,0.354167
4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,M,0.559075,0.783400,0.000026,Dance movement psychotherapist,0.569303,0.797770,0.156250


### Amount
Use the standard MinMaxScaler

In [16]:
from sklearn.preprocessing import MinMaxScaler
X['amt'] = MinMaxScaler().fit_transform(X[['amt']])

X.head()

,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age
0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,0.000137,F,0.464877,0.753721,0.001194,"Psychologist, counselling",0.462162,0.738663,0.135417
1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,0.003670,F,0.980089,0.112922,0.000043,Special educational needs teacher,0.991000,0.113337,0.239583
2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,0.007569,M,0.710316,0.215853,0.001421,Nature conservation officer,0.749328,0.217713,0.406250
3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,0.001520,M,0.873210,0.218417,0.000659,Patent attorney,0.905538,0.210678,0.354167
4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,0.001415,M,0.559075,0.783400,0.000026,Dance movement psychotherapist,0.569303,0.797770,0.156250


## Categorical data

### Gender
The gender can only be either 'F' (for female) or 'M' (for male) so we can use a simple Ordinal Encoder, with 0 for 'F' and 1 for 'M'.

In [17]:
X['gender'].unique()

array(['F', 'M'], dtype=object)

In [18]:
from sklearn.preprocessing import OrdinalEncoder

genders = ['F', 'M']
ord_encoder = OrdinalEncoder(categories = [genders])
X['gender'] = ord_encoder.fit_transform(X[['gender']])
X.head()

,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age
0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,0.000137,0.0,0.464877,0.753721,0.001194,"Psychologist, counselling",0.462162,0.738663,0.135417
1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,0.003670,0.0,0.980089,0.112922,0.000043,Special educational needs teacher,0.991000,0.113337,0.239583
2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,0.007569,1.0,0.710316,0.215853,0.001421,Nature conservation officer,0.749328,0.217713,0.406250
3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,0.001520,1.0,0.873210,0.218417,0.000659,Patent attorney,0.905538,0.210678,0.354167
4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,0.001415,1.0,0.559075,0.783400,0.000026,Dance movement psychotherapist,0.569303,0.797770,0.156250


### Merchant

In [19]:
X['merchant'].nunique()

693

Due to the large amount of unique merchants, it's suboptimal to approch this field the same way we did with genders. Therefore we will use a different technique called "Target encoding".

A target encoding is any kind of encoding that replaces a feature's categories with some number derived from the target. Since our target is unbalanced, we also use "smoothing", of which the idea is to blend the in-category average with the overall average, to avoid overfitting.

In [ ]:
!pip install category_encoders

In [21]:
from category_encoders import MEstimateEncoder

# Choose m to control noise.
encoder = MEstimateEncoder(cols=['merchant'], m=5.0)
encoder.fit(X, y)
X = encoder.transform(X)

In [22]:
X

,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age
0,2019-01-01 00:00:18,0.013551,misc_net,0.000137,0.0,0.464877,0.753721,0.001194,"Psychologist, counselling",0.462162,0.738663,0.135417
1,2019-01-01 00:00:44,0.009914,grocery_pos,0.003670,0.0,0.980089,0.112922,0.000043,Special educational needs teacher,0.991000,0.113337,0.239583
2,2019-01-01 00:00:51,0.001899,entertainment,0.007569,1.0,0.710316,0.215853,0.001421,Nature conservation officer,0.749328,0.217713,0.406250
3,2019-01-01 00:01:16,0.002420,gas_transport,0.001520,1.0,0.873210,0.218417,0.000659,Patent attorney,0.905538,0.210678,0.354167
4,2019-01-01 00:03:06,0.003061,misc_pos,0.001415,1.0,0.559075,0.783400,0.000026,Dance movement psychotherapist,0.569303,0.797770,0.156250
...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,0.002466,health_fitness,0.001477,1.0,0.642432,0.568344,0.000171,Town planner,0.620460,0.577997,0.375000
555715,2020-12-31 23:59:09,0.001830,kids_pets,0.003829,1.0,0.181730,0.506935,0.009879,Futures trader,0.206738,0.494017,0.031250
555716,2020-12-31 23:59:15,0.001828,kids_pets,0.002967,0.0,0.871842,0.100961,0.001260,Musician,0.890415,0.086887,0.218750
555717,2020-12-31 23:59:24,0.002629,travel,0.000241,1.0,0.808648,0.143397,0.000036,Cartographer,0.802415,0.132468,0.385417


### Category
Similar to 'merchant'.

In [23]:
encoder = MEstimateEncoder(cols=['category'], m=5.0)
encoder.fit(X, y)
X = encoder.transform(X)
X

,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age
0,2019-01-01 00:00:18,0.013551,0.013038,0.000137,0.0,0.464877,0.753721,0.001194,"Psychologist, counselling",0.462162,0.738663,0.135417
1,2019-01-01 00:00:44,0.009914,0.012645,0.003670,0.0,0.980089,0.112922,0.000043,Special educational needs teacher,0.991000,0.113337,0.239583
2,2019-01-01 00:00:51,0.001899,0.002177,0.007569,1.0,0.710316,0.215853,0.001421,Nature conservation officer,0.749328,0.217713,0.406250
3,2019-01-01 00:01:16,0.002420,0.004106,0.001520,1.0,0.873210,0.218417,0.000659,Patent attorney,0.905538,0.210678,0.354167
4,2019-01-01 00:03:06,0.003061,0.002819,0.001415,1.0,0.559075,0.783400,0.000026,Dance movement psychotherapist,0.569303,0.797770,0.156250
...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,0.002466,0.001510,0.001477,1.0,0.642432,0.568344,0.000171,Town planner,0.620460,0.577997,0.375000
555715,2020-12-31 23:59:09,0.001830,0.001880,0.003829,1.0,0.181730,0.506935,0.009879,Futures trader,0.206738,0.494017,0.031250
555716,2020-12-31 23:59:15,0.001828,0.001880,0.002967,0.0,0.871842,0.100961,0.001260,Musician,0.890415,0.086887,0.218750
555717,2020-12-31 23:59:24,0.002629,0.002692,0.000241,1.0,0.808648,0.143397,0.000036,Cartographer,0.802415,0.132468,0.385417


### Job

In [24]:
encoder = MEstimateEncoder(cols=['job'], m=5.0)
encoder.fit(X, y)
X = encoder.transform(X)
X

,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age
0,2019-01-01 00:00:18,0.013551,0.013038,0.000137,0.0,0.464877,0.753721,0.001194,0.003322,0.462162,0.738663,0.135417
1,2019-01-01 00:00:44,0.009914,0.012645,0.003670,0.0,0.980089,0.112922,0.000043,0.002473,0.991000,0.113337,0.239583
2,2019-01-01 00:00:51,0.001899,0.002177,0.007569,1.0,0.710316,0.215853,0.001421,0.021425,0.749328,0.217713,0.406250
3,2019-01-01 00:01:16,0.002420,0.004106,0.001520,1.0,0.873210,0.218417,0.000659,0.005461,0.905538,0.210678,0.354167
4,2019-01-01 00:03:06,0.003061,0.002819,0.001415,1.0,0.559075,0.783400,0.000026,0.004450,0.569303,0.797770,0.156250
...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,0.002466,0.001510,0.001477,1.0,0.642432,0.568344,0.000171,0.003762,0.620460,0.577997,0.375000
555715,2020-12-31 23:59:09,0.001830,0.001880,0.003829,1.0,0.181730,0.506935,0.009879,0.005631,0.206738,0.494017,0.031250
555716,2020-12-31 23:59:15,0.001828,0.001880,0.002967,0.0,0.871842,0.100961,0.001260,0.003193,0.890415,0.086887,0.218750
555717,2020-12-31 23:59:24,0.002629,0.002692,0.000241,1.0,0.808648,0.143397,0.000036,0.003768,0.802415,0.132468,0.385417


# Split the dataset for training and testing

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

### Output the data to csv for future uses

In [35]:
data_train = pd.merge(X_train.reset_index(), y_train.reset_index(), left_index = True, right_index = True)
data_train

,index_x,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age,index_y,is_fraud
0,1004645,2020-02-16 19:12:05,0.014663,0.015927,0.003125,1.0,0.913292,0.132451,0.000300,0.004336,0.930886,0.143355,0.031250,1004645,0
1,491913,2020-12-17 21:41:36,0.002665,0.002229,0.000752,0.0,0.651832,0.008980,0.000087,0.004495,0.619573,0.019476,0.760417,491913,0
2,160547,2019-03-25 23:51:31,0.002279,0.001568,0.001534,1.0,0.429365,0.707958,0.001834,0.006828,0.456351,0.710208,0.375000,160547,0
3,464288,2019-07-23 19:57:58,0.001234,0.001880,0.003678,0.0,0.665419,0.642742,0.000727,0.003983,0.661527,0.628499,-0.031250,464288,0
4,564342,2019-08-28 09:37:57,0.006705,0.006344,0.019995,1.0,0.418661,0.687787,0.000018,0.004107,0.429634,0.694310,0.593750,564342,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481910,491263,2019-08-02 23:40:30,0.001706,0.001510,0.001297,0.0,0.631592,0.834012,0.000065,0.002473,0.630164,0.835532,0.302083,491263,0
1481911,543525,2020-12-29 01:53:39,0.001173,0.001568,0.002779,0.0,0.470561,0.572654,0.000975,0.008149,0.454398,0.581218,0.791667,543525,0
1481912,470924,2019-07-26 19:30:01,0.002192,0.001880,0.005955,1.0,0.080759,0.737914,0.001291,0.008155,0.046839,0.750249,0.260417,470924,0
1481913,491755,2019-08-03 03:45:26,0.012381,0.012645,0.004280,0.0,0.670660,0.610743,0.000251,0.010836,0.706651,0.609433,0.406250,491755,0


In [33]:
data_test = pd.merge(X_test.reset_index(), y_test.reset_index(), left_index = True, right_index = True)
data_test

,index_x,trans_date_trans_time,merchant,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,age,index_y,is_fraud
0,519669,2019-08-12 08:58:23,0.014377,0.012645,0.003954,1.0,0.656040,0.705310,0.000112,0.008837,0.619587,0.706820,0.239583,519669,0
1,154807,2020-08-14 16:16:11,0.001144,0.001568,0.000546,1.0,0.551613,0.347396,0.052222,0.007159,0.564006,0.354621,0.145833,154807,0
2,186902,2020-08-25 14:18:17,0.002547,0.002692,0.000307,1.0,0.544003,0.044120,0.000060,0.008166,0.523579,0.038152,0.447917,186902,0
3,406717,2020-11-28 21:30:01,0.013589,0.013038,0.000257,0.0,0.807603,0.666579,0.000120,0.003426,0.779541,0.666266,0.072917,406717,0
4,383569,2019-06-24 20:27:38,0.003257,0.002692,0.000117,1.0,0.977877,0.282747,0.000058,0.007159,0.973691,0.286969,0.385417,383569,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370474,152875,2020-08-13 18:23:08,0.007991,0.006344,0.000514,0.0,0.229965,0.586473,0.003459,0.002474,0.270184,0.588785,0.260417,152875,0
370475,301248,2020-10-12 21:50:39,0.004496,0.006344,0.005763,0.0,0.672551,0.743906,0.002623,0.001371,0.677635,0.749804,0.145833,301248,0
370476,832249,2019-12-11 12:48:20,0.003107,0.002229,0.006059,0.0,0.470561,0.572654,0.000975,0.008149,0.478551,0.567057,0.781250,832249,0
370477,448800,2019-07-18 19:13:13,0.002298,0.001510,0.005894,0.0,0.980089,0.112922,0.000043,0.002473,0.960441,0.109982,0.239583,448800,0


In [36]:
data_test.drop(columns = ['index_x', 'index_y']).to_csv('data3_test.csv')

In [37]:
data_train.drop(columns = ['index_x', 'index_y']).to_csv('data3_train.csv')

In [38]:
data_test['is_fraud'].value_counts()

0    368534
1      1945
Name: is_fraud, dtype: int64

In [39]:
data_train['is_fraud'].value_counts()

0    1474209
1       7706
Name: is_fraud, dtype: int64